# Introduction - Really Simple Getting Started Sample

This polyglot will run load a Phi-3 model and run inference on it. The Phi-3 model is a mini model that is designed to be used on a CPU or mobile device.

> Note: This sample rather than uses AI Toolkit (AITK) to interact with the Phi3 model. Interacts with the Phi3 model directly - allowing for flexibility and reduced dependencies on future learning.

 *  This is based on the Phi3 model from Microsoft. The Phi3 model is a mini model that is designed to be used on a CPU or mobile device.
 *  Original Sample: https://github.com/microsoft/Phi-3CookBook from Labs.

In [ ]:
#r "nuget: Microsoft.ML.OnnxRuntime, 1.18.1"
#r "nuget: Microsoft.ML.OnnxRuntimeGenAI, 0.3.0"
using Microsoft.ML.OnnxRuntimeGenAI;
using System.IO;

In [ ]:
// Forces the method to be execute synchronously
public void ProcessPrompt(string userPrompt, string systemPrompt){

    // Download models with the AI Toolkit and reference this location
    var modelPath = @"C:\Git\readonly\Phi-3-medium-128k-instruct-onnx-cpu\cpu-int4-rtn-block-32-acc-level-4";
    Console.WriteLine("Model Loaded: {0}", Path.Exists(modelPath) ? "Yes" : "No");
    var model = new Model(modelPath);
    var tokenizer = new Tokenizer(model);

    Console.WriteLine("User: " + userPrompt);

    // show phi3 response
    Console.Write("Phi3: ");
    var fullPrompt = $"<|system|>{systemPrompt}<|end|><|user|>{userPrompt}<|end|><|assistant|>";
    var tokens = tokenizer.Encode(fullPrompt);

    var generatorParams = new GeneratorParams(model);
    generatorParams.SetSearchOption("max_length", 2048);
    generatorParams.SetSearchOption("past_present_share_buffer", false);
    generatorParams.SetInputSequences(tokens);

    var generator = new Generator(model, generatorParams);
    while (!generator.IsDone())
    {
        generator.ComputeLogits();
        generator.GenerateNextToken();
        ReadOnlySpan<int> outputTokens = generator.GetSequence(0);
        ReadOnlySpan<int> newToken = outputTokens.Slice(outputTokens.Length - 1, 1);
        var output = tokenizer.Decode(newToken);
        Console.Write(output);
    }
}

var userPrompt = "Write a PowerShell script for getting items on the file system.";
var systemPrompt = "You are an AI assistant that helps people find information. Answer questions using a direct style. Do not share more information that the requested by the users.";

ProcessPrompt(userPrompt, systemPrompt);

Console.WriteLine("\nDone :-)");